In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import time
import nni.retiarii.nn.pytorch as nn
from nni.retiarii import model_wrapper
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

import pandas as pd
import json
import os

# Device configuration
#
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Check the number of available GPUs
num_gpus = torch.cuda.device_count()

if num_gpus >= 2:
    # Select the second GPU (index 1)
    device = torch.device("cuda:1")
else:
    # If there are not enough GPUs, use the first one (index 0) or CPU
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [2]:
import nni.retiarii.nn.pytorch as nn
from nni.retiarii import model_wrapper

@model_wrapper
class ResNet_(nn.Module):
    def __init__(self, num_channel_1, kernel_sizes, strides, paddings, kernel_size_pool, stride_size_pool, poolchoice, num_classes=2):
        super().__init__()
        self.model_name = 'resnet'
        self.num_classes = num_classes
        self.conv1 = nn.Conv2d(NUM_FEATURES, num_channel_1, kernel_size=kernel_sizes, stride=strides, padding=paddings, bias=False)
        self.bn1 = nn.BatchNorm2d(num_channel_1)
        self.relu = nn.ReLU(inplace=True)
        
        self.poolchoice = [nn.MaxPool2d(kernel_size=kernel_size_pool, stride=stride_size_pool, padding=1),
                           nn.Identity()][poolchoice]
        
        num_channel_2 = num_channel_1*2
        num_channel_3 = num_channel_2*2
        num_channel_4 = num_channel_3*2

        ### layer1
        self.block1 = self.make_block(num_channel_1, num_channel_1, 1)
        self.block2 = self.make_block(num_channel_1, num_channel_1, 1)
        ### layer2
        self.block3 = self.make_block(num_channel_1, num_channel_2, 2)
        self.block4 = self.make_block(num_channel_2, num_channel_2, 1)
        ### layer3
        self.block5 = self.make_block(num_channel_2, num_channel_3, 2)
        self.block6 = self.make_block(num_channel_3, num_channel_3, 1)
        ### layer4
        self.block7 = self.make_block(num_channel_3, num_channel_4, 2)
        self.block8 = self.make_block(num_channel_4, num_channel_4, 1)
        
        ### downsample
        self.ds2 = nn.Sequential(nn.Conv2d(num_channel_1, num_channel_2, kernel_size=1, stride=2, bias=False),
                                 nn.BatchNorm2d(num_channel_2))
        self.ds3 = nn.Sequential(nn.Conv2d(num_channel_2, num_channel_3, kernel_size=1, stride=2, bias=False),
                                 nn.BatchNorm2d(num_channel_3))
        self.ds4 = nn.Sequential(nn.Conv2d(num_channel_3, num_channel_4, kernel_size=1, stride=2, bias=False),
                                 nn.BatchNorm2d(num_channel_4))
        
        self.adaptivepool = nn.AdaptiveAvgPool2d((1, 1))

        self.fc = nn.Linear(num_channel_4, num_classes)

    def make_block(self, inplanes, planes, stride=1):
        conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        bn1 = nn.BatchNorm2d(planes)
        relu = nn.ReLU(inplace=True)
        conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        bn2 = nn.BatchNorm2d(planes)
        basicblock = []
        basicblock += [conv1, bn1, relu, conv2, bn2]
        return nn.Sequential(*basicblock)
           
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.poolchoice(x)
        x1 = self.block1(x) + x
        x1 = self.block2(x1) + x1
        x2 = self.ds2(x1) + self.block3(x1)
        x2 = self.block4(x2) + x2
        x3 = self.ds3(x2) + self.block5(x2)
        x3 = self.block6(x3) + x3
        x4 = self.ds4(x3) + self.block7(x3)
        x4 = self.block8(x4) + x4
        x = self.adaptivepool(x4)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
        
        
        

In [ ]:
import nn_meter
print(f"nn_meter version: {nn_meter.__version__}")

# list all supporting latency predictors
predictors = nn_meter.list_latency_predictors()
for p in predictors:
    print(f"[Predictor] {p['name']}: version={p['version']}")
predictor_names = [p['name'] for p in predictors]
predictors = [nn_meter.load_latency_predictor(p['name'], 1.0) for p in predictors]

In [4]:
def check_latency(re_path, in_channels, batch_s, predictor):
    nni_re = pd.read_csv(re_path)
    input_shape = (batch_s, in_channels, 100, 100)

    lats = np.zeros(len(nni_re))
    sizes = np.zeros(len(nni_re))
    for i in range(len(nni_re)):
        t = nni_re.mutation[i].replace("'", "\"")
        d = json.loads(t)
        model = ResNet_(num_channel_1=d['num_channel'], 
                        kernel_sizes=d['kernel_size'], 
                        strides=d['stride'], 
                        paddings=d['padding'], 
                        kernel_size_pool=d['kernel_size_pool'], 
                        stride_size_pool=d['stride_size_pool'], 
                        poolchoice=int(d['poolchoice'])).to(device)

        dummy_input = torch.randn(batch_s, in_channels, 100, 100).to(device)
        torch.onnx.export(model, dummy_input, "temp.onnx")
        size = os.path.getsize("temp.onnx")
        sizes[i] = size
        
        lat = predictor.predict(model, model_type='torch', input_shape=input_shape, apply_nni=False) 
        lats[i] = lat
    return (lats, sizes)



In [ ]:
all_re = pd.DataFrame()

lats = []
sizes = []

for re_path in ["7ch-all-lnvbs15o.csv", "5ch-all-20fh3z74.csv", "7ch-8batch-z0dv9mrl.csv", "5ch-8batch-lwpxanm3.csv", "5ch-32batch-qt94sfe8.csv", "7ch-32batch-0v3gpsxz.csv"]:
        lats = []
        sizes = []
        for p, predictor in enumerate(predictors):
            # load predictor
            NUM_FEATURES = int(re_path[0])
            
            tp = re_path.split("-")[1]
            if tp[0] == "a":
                batch_s = 16
            elif tp[0] == "3":
                batch_s = 32
            elif tp[0] == '8':
                batch_s = 8
            else:
                print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                break
            lats, sizes = check_latency(re_path, NUM_FEATURES, batch_s, predictor)        
        
            re = pd.read_csv(re_path)
            print("-----------------------------", len(re))
            re["features"] = NUM_FEATURES
            re["batch_s"] = batch_s
            re["acc"] = re.reward
            re["latency"] = lats
            re["lat_std"] = lat_stds
            re["memory"] = sizes
            re["predictor"] = predictor_names[p]

            all_re = pd.concat([all_re, re]).reset_index(drop=True)
            print("-------------------------------------------------", len(all_re))

In [6]:
re = all_re
re["memory"] = re["memory"]/1e6
t = re.mutation[0].replace("'", "\"")
d = json.loads(t)
dicts = re.mutation.apply(lambda x: json.loads(x.replace("'", "\"")))

for col in d.keys():
    re[col] = [dt[col] for dt in dicts]
re_fin = re[['trialJobId', 'features', 'batch_s', 'acc', 'latency', 'memory', 'predictor'] + list(d.keys())]
re_fin.to_csv("fin_results.csv")